In [9]:
# -*- coding: utf-8 -*-
"""
Created on Fri Jun 11 11:02:03 2021

@author: Christian
"""

from mip import Model, xsum, maximize, BINARY

from Co2_price import df_data
from electricity_demand import electricity_demand
from Electricity_Price0 import Electricity_price_pivot0
from Electricity_Price1 import Electricity_price_pivot1
from gas_price import gas_p
from heat_demand import heat_demand_pivot
from heat_price import heat_p
from elec_effciency import elec_eff
from electricity_capacity import elec_capacity
from heat_capacity import heat_capacity


In [16]:


elec_demand = electricity_demand.stack().tolist()
elec_price_wholesale = Electricity_price_pivot0
elec_price_household_32_05 = Electricity_price_pivot1
co2_price = df_data
gas_price = gas_p
heat_demand_1 = heat_demand_pivot
heat_demand_1 = heat_demand_1.interpolate(method ='linear', limit_direction ='forward')
heat_price_1 = heat_p


el_price = elec_price_wholesale.stack().tolist() # Electricity price
del_t = .25 # duration of time step
el_demand = electricity_demand.stack().tolist() # Electicity demand


gas_pp = gas_price["Preis"].values.tolist() # Gas price for different plants
em_fc = 0.2 # emission factor
co2_p = co2_price["mean_CO2_tax"].values.tolist() # CO2 Price
capacity_el = elec_capacity.stack().tolist() # maximum capacity of electricity for the power plants
capacity_ht = heat_capacity.stack().tolist() # maximum capacity of heat for the power plants
heat_demand = heat_demand_1.stack().tolist() # heat demand
heat_price = heat_price_1.stack().tolist() # heat price
heat_ratio = 400/385 # heat ratio


eff_plants = elec_eff.stack().tolist() # the efficiency of the power plants


T = range(8784)
I = range(1)

m = Model("Maximizing profit", sense=maximize)
 

#variable
y_t = [[m.add_var(lb = 0) for i in I] for t in T] # Electricity generation
el_sold = [[m.add_var(lb = 0) for i in I] for t in T] # electricity sold 
# el_sold = [m.add_var(lb = 0) for t in T] # electricity sold 
x_t = [[m.add_var(lb = 0) for i in I] for t in T] # Fuel consumption
z_t = [[m.add_var(lb = 0) for i in I] for t in T] # Heat generation
# objective function

# Revenue = Electricity price * demand/generation
# Cost = Fuel price = gas price + emission_factor * co2 price

# Max Proft = Revenue - Cost



In [17]:
heat_demand_1

hour,00:00:00,01:00:00,02:00:00,03:00:00,04:00:00,05:00:00,06:00:00,07:00:00,08:00:00,09:00:00,...,14:00:00,15:00:00,16:00:00,17:00:00,18:00:00,19:00:00,20:00:00,21:00:00,22:00:00,23:00:00
date,,,,,,,,,,,,,,,,,,,,,
2012-01-01,47077000.0,59491000.0,57734000.0,62913000.0,77677000.0,105238000.0,114154000.0,107996000.0,106766000.0,101548000.0,...,93100000.0,95736000.0,100419000.0,105237000.0,107291000.0,106845000.0,104586000.0,94116000.0,75726000.0,61090000.0
2012-01-02,52144000.0,49741000.0,50926000.0,56776000.0,68749000.0,86988000.0,100699000.0,99627000.0,99663000.0,92958000.0,...,82832000.0,85758000.0,89926000.0,93025000.0,96041000.0,95494000.0,91746000.0,82862000.0,64275000.0,51682000.0
2012-01-03,54257000.0,54816000.0,56107000.0,62414000.0,75600000.0,96777000.0,112554000.0,111367000.0,111580000.0,103283000.0,...,92399000.0,95435000.0,99348000.0,102940000.0,104696000.0,104469000.0,100877000.0,91066000.0,70257000.0,56659000.0
2012-01-04,59315000.0,57095000.0,58455000.0,65057000.0,78889000.0,101300000.0,118014000.0,116697000.0,116661000.0,107928000.0,...,96365000.0,99506000.0,103556000.0,107369000.0,109208000.0,109043000.0,105306000.0,95095000.0,73242000.0,59015000.0
2012-01-05,68047000.0,62181000.0,63678000.0,70265000.0,84014000.0,108208000.0,125904000.0,124340000.0,124283000.0,116018000.0,...,104023000.0,107197000.0,111498000.0,115099000.0,116679000.0,116125000.0,112165000.0,101370000.0,78775000.0,63980000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012-12-27,62817000.0,51119000.0,52314000.0,58341000.0,70888000.0,90369000.0,104979000.0,103984000.0,104434000.0,96416000.0,...,86330000.0,89266000.0,92961000.0,96403000.0,98152000.0,97978000.0,94577000.0,85360000.0,65711000.0,52924000.0
2012-12-28,56028000.0,65716000.0,67460000.0,73008000.0,85646000.0,110792000.0,131153000.0,128798000.0,128355000.0,121347000.0,...,109804000.0,112832000.0,117162000.0,119758000.0,121384000.0,119902000.0,115652000.0,104778000.0,82808000.0,67942000.0
2012-12-29,55802000.0,58145000.0,60682000.0,66008000.0,79033000.0,105253000.0,120440000.0,116332000.0,115443000.0,108885000.0,...,93562000.0,97701000.0,102827000.0,108166000.0,110802000.0,109674000.0,106370000.0,96409000.0,76932000.0,62204000.0


In [18]:
# heat_demand_1.isnull()


# heat_demand_1.interpolate(method ='linear', limit_direction ='forward')

In [19]:
heat_demand_norm = [((element/max(heat_demand))*385) for element in heat_demand] 


In [20]:
print(len(co2_price))
print(len(el_demand))
print(len(gas_pp))
print(len(co2_p))
print(len(capacity_el))
print(len(capacity_ht))
print(len(heat_demand))
print(len(heat_price))
print(len(eff_plants))
print(len(heat_demand_norm))





366
8784
366
366
8784
8784
8784
8784
8784
8784


In [21]:
# m.objective = xsum((el_price[t] * del_t * el_sold[t]) - ( x_t[t][i] * ( gas_p[i] * del_t + (em_fc * co2_p[t]*del_t))) for t in T for i in I) + ((heat_demand[t] * heat_price[t]) for t in T)


In [22]:
m.objective = xsum((el_price[t] * del_t * el_sold[t]) - ( x_t[t][i] * ( gas_p[i] * del_t + (em_fc * co2_p[t]*del_t))) for t in T for i in I) + ((heat_demand[t] * heat_price[t]) for t in T)


TypeError: can't multiply sequence by non-int of type 'float'

In [2]:
m.objective = xsum((el_price[t] * del_t * el_sold[t]) - ( x_t[t][i] * ( gas_p[i] * del_t + (em_fc * co2_p[t]*del_t))) for t in T for i in I) + ((heat_demand[t] * heat_price[t]) for t in T)

# constraints

for t in T:
    m += xsum(y_t[t][i] for i in I) == el_demand[t]
    
    for i in I:
        m += y_t[t][i] <= capacity_el[i] # electricity generation <= maximum capacity of electricity of the plant

        m += z_t[t][i] <= capacity_ht[i] # heat generation <= maximum capacity of heat of the plant 
        
        m += x_t[t][i] == y_t[t][i]/eff_plants[i] # fuel consumption = Electricity generation / efficiency of the plants 

        m += heat_demand[t] <= z_t[t][i] # heat demand >= Heat generation  

        m += y_t[t][i] == heat_ratio * z_t[t][i] # electricity generation >= heat ratio * heat generation

        m += y_t[t][i] == el_sold[t] + el_demand[t]



status = m.optimize()
obj= m.objective_value

41.709999999999994

In [3]:
status

<OptimizationStatus.OPTIMAL: 0>

In [7]:
r = [3, 5, 8, 3, 6, 5, 7, 3]


In [8]:
type(r)

list

In [6]:
from electricity_demand import electricity_demand
import numpy as np
x = electricity_demand.stack().tolist()
x

[63.974999999999994,
 54.150000000000006,
 49.075,
 48.625,
 56.7,
 65.4,
 75.4,
 103.625,
 168.375,
 225.85,
 231.02499999999998,
 238.5,
 228.02499999999998,
 191.8,
 172.05,
 185.2,
 199.95,
 204.275,
 182.175,
 126.87500000000001,
 98.85,
 86.85,
 77.25,
 72.8,
 63.974999999999994,
 54.150000000000006,
 49.075,
 48.625,
 56.7,
 65.4,
 75.4,
 103.625,
 168.375,
 225.85,
 231.02499999999998,
 238.5,
 228.02499999999998,
 191.8,
 172.05,
 185.2,
 199.95,
 204.275,
 182.175,
 126.87500000000001,
 98.85,
 86.85,
 77.25,
 72.8,
 63.974999999999994,
 54.150000000000006,
 49.075,
 48.625,
 56.7,
 65.4,
 75.4,
 103.625,
 168.375,
 225.85,
 231.02499999999998,
 238.5,
 228.02499999999998,
 191.8,
 172.05,
 185.2,
 199.95,
 204.275,
 182.175,
 126.87500000000001,
 98.85,
 86.85,
 77.25,
 72.8,
 71.075,
 75.44999999999999,
 66.27499999999999,
 63.625,
 65.35,
 69.275,
 76.275,
 105.325,
 158.375,
 190.37500000000003,
 201.24999999999997,
 207.875,
 200.975,
 180.4,
 140.875,
 101.475,
 85.025,

In [7]:
len(x)

8784